# The following is are examples of using Laplacians/primalDualIPM for flow problems

In [22]:
using Laplacians
include("/Users/spielman/Laplacians/src/min_cost_flow.jl")
include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")
include("/Users/spielman/Laplacians/src/flowUtils.jl")

cutCapacity

## First, we try some max flow problems.  
This code generates some, and then calls max_flow_IPM.
We will start small.

In [4]:
n = 100
a = chimera(n,1);
af,s,t = makeSTFlowProblem(a)
fl = stFlow(af,s,t);

size(Bt) = (102,396)
Ratio of edge weights: 1.0
Iteration 1, ||r_p||=63.635231, ||r_d||=14.726782, mu=1.235272
Ratio of edge weights: 3.517153857092042
sigma parameter is 0.13303917704244886
Iteration 2, ||r_p||=0.636352, ||r_d||=3.530600, mu=0.548329
Ratio of edge weights: 32.57177259857809
sigma parameter is 1.7951541493146115e-6
Iteration 3, ||r_p||=0.006364, ||r_d||=0.035306, mu=0.005497
Ratio of edge weights: 27.946566721935348
sigma parameter is 0.17360746099270727
Iteration 4, ||r_p||=0.000064, ||r_d||=0.002808, mu=0.001312
Ratio of edge weights: 595.8061504586377
sigma parameter is 0.05445096229748747
Iteration 5, ||r_p||=0.000013, ||r_d||=0.000415, mu=0.000259
Ratio of edge weights: 183939.96060825256
sigma parameter is 0.021582662931177308
Iteration 6, ||r_p||=0.000001, ||r_d||=0.000059, mu=0.000041
Ratio of edge weights: 5.132822274923839e7
sigma parameter is 0.0011419904911829245
Iteration 7, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000001
Termination tolerance reached.

Th

Now, let's try a weighted graph.

In [5]:
n = 100
a = wtedChimera(n,1);
af,s,t = makeSTFlowProblem(a)
fl = stFlow(af,s,t);

size(Bt) = (102,396)
Ratio of edge weights: 1.0
Iteration 1, ||r_p||=1426.380367, ||r_d||=287.454810, mu=81.136838
Ratio of edge weights: 162.45982209158493
sigma parameter is 0.5496894795824239
Iteration 2, ||r_p||=1225.714671, ||r_d||=215.090875, mu=80.285022
Ratio of edge weights: 66105.1351595706
sigma parameter is 0.26759509789060204
Iteration 3, ||r_p||=454.807757, ||r_d||=79.894332, mu=46.564543
Ratio of edge weights: 3559.3573648212396
sigma parameter is 0.11973211223177334
Iteration 4, ||r_p||=116.762929, ||r_d||=7.348468, mu=12.416308
Ratio of edge weights: 19645.86693280167
sigma parameter is 0.06810841630934861
Iteration 5, ||r_p||=1.167629, ||r_d||=0.647778, mu=2.048510
Ratio of edge weights: 6549.513979529065
sigma parameter is 4.523184885344383e-5
Iteration 6, ||r_p||=0.099771, ||r_d||=0.013154, mu=0.041790
Ratio of edge weights: 6900.899938439309
sigma parameter is 0.00026565770010078196
Iteration 7, ||r_p||=0.000998, ||r_d||=0.000661, mu=0.002109
Ratio of edge weights:

The gap between the flow and cut above should be a bit of a concern.
It could be due to our crude heuristic for computing the cut.
But, below we see that there is flow out of t, which I don't think should happen.

In [13]:
f = fl[3]
(afi,afj) = findnz(af)
@show sum(f[afj.==t])
@show sum(f[afi.==t])


sum(f[afj .== t]) = 12.134818765108266
sum(f[afi .== t]) = 17.885040153054387


17.885040153054387

Let's try a much larger unweighed graph.  We seem to do better with the unweighted ones. We might need to use a better solver.  By default, the code so far was using cholesky, which can be slow.

In [27]:
n = 10000
a = chimera(n,1);
af,s,t = makeSTFlowProblem(a)
#fl = stFlow(af,s,t,tol=1e-6,lapSolver = (h->augTreeLap(h,tol=1e-6)));
fl = stFlow(af,s,t,tol=1e-6);

size(Bt) = (10002,51440)
Ratio of edge weights: 1.0
Iteration 1, ||r_p||=508.827755, ||r_d||=96.668490, mu=0.715187
Ratio of edge weights: 1.0000189769883967
sigma parameter is 0.01122574491220749
Iteration 2, ||r_p||=5.088278, ||r_d||=0.966685, mu=0.066342
Ratio of edge weights: 15.747162877799266
sigma parameter is 3.8968149502740955e-6
Iteration 3, ||r_p||=0.050883, ||r_d||=0.009667, mu=0.000662
Ratio of edge weights: 15.908080576054914
sigma parameter is 0.41108679484326427
Iteration 4, ||r_p||=0.000509, ||r_d||=0.005604, mu=0.000498
Ratio of edge weights: 19.850198902017333
sigma parameter is 0.28023057338185947
Iteration 5, ||r_p||=0.000005, ||r_d||=0.000056, mu=0.000143
Ratio of edge weights: 107.17360006347366
sigma parameter is 0.17902105367615506
Iteration 6, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000027
Ratio of edge weights: 519890.14570793277
sigma parameter is 0.11656530241121132
Iteration 7, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000009
Ratio of edge weights: 433188.

This worked, but it is very finnicky.  Using a lower tol can cause it to die.  This is probably because the ratio of the edge weights grows huge.  The following is an example on which it dies.

In [28]:
fl = stFlow(af,s,t,tol=1e-8);

size(Bt) = (10002,51440)
Ratio of edge weights: 1.0
Iteration 1, ||r_p||=508.827755, ||r_d||=96.668490, mu=0.715187
Ratio of edge weights: 1.0000189769883967
sigma parameter is 0.01122574491220749
Iteration 2, ||r_p||=5.088278, ||r_d||=0.966685, mu=0.066342
Ratio of edge weights: 15.747162877799266
sigma parameter is 3.8968149502740955e-6
Iteration 3, ||r_p||=0.050883, ||r_d||=0.009667, mu=0.000662
Ratio of edge weights: 15.908080576054914
sigma parameter is 0.41108679484326427
Iteration 4, ||r_p||=0.000509, ||r_d||=0.005604, mu=0.000498
Ratio of edge weights: 19.850198902017333
sigma parameter is 0.28023057338185947
Iteration 5, ||r_p||=0.000005, ||r_d||=0.000056, mu=0.000143
Ratio of edge weights: 107.17360006347366
sigma parameter is 0.17902105367615506
Iteration 6, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000027
Ratio of edge weights: 519890.14570793277
sigma parameter is 0.11656530241121132
Iteration 7, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000009
Ratio of edge weights: 433188.

LoadError: Base.LinAlg.PosDefException(10000)

Now, let's compare our answer with the one given by our other maxflow code.

In [24]:
chf = Laplacians.maxflow(af,s,t)

98.0

## Now, let's try testing the min cost flow code.
I don't have any other code that computes this, so I won't know if the answer is correct.
But, at least we can get some timings.

In [35]:
n = 100
a = chimera(n,1)

Bt= -dirEdgeVertexMat(a)';
m = nnz(a)

c = rand(m);
b = 0.25*Bt*rand(m);
u = 0.25*ones(m);


In [36]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = cholLap);

Iteration 1, ||r_p||=1.806108, ||r_d||=31.920958, mu=0.687489
Iteration 2, ||r_p||=0.622969, ||r_d||=11.010282, mu=0.438648
Iteration 3, ||r_p||=0.130696, ||r_d||=2.309908, mu=0.102424
Iteration 4, ||r_p||=0.039113, ||r_d||=0.691285, mu=0.045795
Iteration 5, ||r_p||=0.008959, ||r_d||=0.158333, mu=0.012627
Iteration 6, ||r_p||=0.002263, ||r_d||=0.039992, mu=0.005137
Iteration 7, ||r_p||=0.000119, ||r_d||=0.002111, mu=0.001967
Iteration 8, ||r_p||=0.000019, ||r_d||=0.000331, mu=0.000554
Iteration 9, ||r_p||=0.000003, ||r_d||=0.000046, mu=0.000161
Iteration 10, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000041
Iteration 11, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000007
Iteration 12, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000000
Termination tolerance reached.
  0.118599 seconds (33.48 k allocations: 10.960 MB)


Let's run on a much larger example

In [39]:
n = 10000
a = chimera(n,1)

Bt= -dirEdgeVertexMat(a)';
m = nnz(a)

c = rand(m);
b = 0.25*Bt*rand(m);
u = 0.25*ones(m);

@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = cholLap);

Iteration 1, ||r_p||=22.992661, ||r_d||=408.282072, mu=0.777585
Iteration 2, ||r_p||=10.735538, ||r_d||=190.631589, mu=0.590004
Iteration 3, ||r_p||=2.344975, ||r_d||=41.639870, mu=0.153289
Iteration 4, ||r_p||=0.646444, ||r_d||=11.478952, mu=0.069290
Iteration 5, ||r_p||=0.163679, ||r_d||=2.906466, mu=0.023968
Iteration 6, ||r_p||=0.038889, ||r_d||=0.690546, mu=0.009859
Iteration 7, ||r_p||=0.005667, ||r_d||=0.100622, mu=0.004606
Iteration 8, ||r_p||=0.001283, ||r_d||=0.022791, mu=0.001969
Iteration 9, ||r_p||=0.000186, ||r_d||=0.003297, mu=0.000869
Iteration 10, ||r_p||=0.000027, ||r_d||=0.000485, mu=0.000282
Iteration 11, ||r_p||=0.000007, ||r_d||=0.000117, mu=0.000137
Iteration 12, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000060
Iteration 13, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000024
Iteration 14, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000010
Iteration 15, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000003
Iteration 16, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000001
Iteration

Now, let's see what happens when we try different solvers.
If you are running this, be sure to run twice, as it compiles the first time.

In [40]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = augTreeLap);

Iteration 1, ||r_p||=22.992661, ||r_d||=408.282072, mu=0.777585
Iteration 2, ||r_p||=10.735542, ||r_d||=190.631672, mu=0.590004
Iteration 3, ||r_p||=2.344979, ||r_d||=41.639928, mu=0.153289
Iteration 4, ||r_p||=0.646445, ||r_d||=11.478973, mu=0.069290
Iteration 5, ||r_p||=0.163679, ||r_d||=2.906470, mu=0.023968
Iteration 6, ||r_p||=0.038888, ||r_d||=0.690547, mu=0.009859
Iteration 7, ||r_p||=0.005667, ||r_d||=0.100622, mu=0.004606
Iteration 8, ||r_p||=0.001283, ||r_d||=0.022791, mu=0.001969
Iteration 9, ||r_p||=0.000186, ||r_d||=0.003297, mu=0.000869
Iteration 10, ||r_p||=0.000027, ||r_d||=0.000485, mu=0.000282
Iteration 11, ||r_p||=0.000007, ||r_d||=0.000117, mu=0.000137
Iteration 12, ||r_p||=0.000002, ||r_d||=0.000001, mu=0.000060
Iteration 13, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000024
Iteration 14, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000010
Iteration 15, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000003
Iteration 16, ||r_p||=0.000001, ||r_d||=0.000000, mu=0.000001
Iteration

In [45]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = (h->KMPLapSolver(h,tol=1e-8)));

Iteration 1, ||r_p||=22.992661, ||r_d||=408.282072, mu=0.777585
Iteration 2, ||r_p||=10.735538, ||r_d||=190.631589, mu=0.590004
Iteration 3, ||r_p||=2.344975, ||r_d||=41.639869, mu=0.153289
Iteration 4, ||r_p||=0.646444, ||r_d||=11.478951, mu=0.069290
Iteration 5, ||r_p||=0.163679, ||r_d||=2.906466, mu=0.023968
Iteration 6, ||r_p||=0.038889, ||r_d||=0.690546, mu=0.009859
Iteration 7, ||r_p||=0.005667, ||r_d||=0.100622, mu=0.004606
Iteration 8, ||r_p||=0.001283, ||r_d||=0.022791, mu=0.001969
Iteration 9, ||r_p||=0.000186, ||r_d||=0.003297, mu=0.000869
Iteration 10, ||r_p||=0.000027, ||r_d||=0.000485, mu=0.000282
Iteration 11, ||r_p||=0.000007, ||r_d||=0.000117, mu=0.000137
Iteration 12, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000060
Iteration 13, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000024
Iteration 14, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000010
Iteration 15, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000003
Iteration 16, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000001
Iteration

Note that if I ran that with a lower tolerance on the solve, then we have trouble with the convergence of the IPM.

In [47]:
n = 50000
a = chimera(n,1)

Bt= -dirEdgeVertexMat(a)';
m = nnz(a)

c = rand(m);
b = 0.25*Bt*rand(m);
u = 0.25*ones(m);

@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = augTreeLap);

Iteration 1, ||r_p||=45.808803, ||r_d||=826.954312, mu=0.804767
Iteration 2, ||r_p||=24.810425, ||r_d||=447.885279, mu=0.664281
Iteration 3, ||r_p||=3.519400, ||r_d||=63.533277, mu=0.166916
Iteration 4, ||r_p||=0.901392, ||r_d||=16.272197, mu=0.071359
Iteration 5, ||r_p||=0.218615, ||r_d||=3.946510, mu=0.026210
Iteration 6, ||r_p||=0.052552, ||r_d||=0.948694, mu=0.011219
Iteration 7, ||r_p||=0.010869, ||r_d||=0.196206, mu=0.005185
Iteration 8, ||r_p||=0.002098, ||r_d||=0.037867, mu=0.002788
Iteration 9, ||r_p||=0.000119, ||r_d||=0.002149, mu=0.001118
Iteration 10, ||r_p||=0.000016, ||r_d||=0.000265, mu=0.000459
Iteration 11, ||r_p||=0.000004, ||r_d||=0.000035, mu=0.000275
Iteration 12, ||r_p||=0.000005, ||r_d||=0.000011, mu=0.000112
Iteration 13, ||r_p||=0.000005, ||r_d||=0.000001, mu=0.000050
Iteration 14, ||r_p||=0.000005, ||r_d||=0.000000, mu=0.000022
Iteration 15, ||r_p||=0.000004, ||r_d||=0.000000, mu=0.000010
Iteration 16, ||r_p||=0.000005, ||r_d||=0.000000, mu=0.000004
Iteration

In [48]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = (h->KMPLapSolver(h,tol=1e-8)));

Iteration 1, ||r_p||=45.808803, ||r_d||=826.954312, mu=0.804767
Iteration 2, ||r_p||=24.810426, ||r_d||=447.885281, mu=0.664281
Iteration 3, ||r_p||=3.519378, ||r_d||=63.532867, mu=0.166915
Iteration 4, ||r_p||=0.901387, ||r_d||=16.272114, mu=0.071359
Iteration 5, ||r_p||=0.218614, ||r_d||=3.946492, mu=0.026210
Iteration 6, ||r_p||=0.052552, ||r_d||=0.948689, mu=0.011219
Iteration 7, ||r_p||=0.010869, ||r_d||=0.196205, mu=0.005185
Iteration 8, ||r_p||=0.002098, ||r_d||=0.037867, mu=0.002788
Iteration 9, ||r_p||=0.000119, ||r_d||=0.002149, mu=0.001118
Iteration 10, ||r_p||=0.000015, ||r_d||=0.000265, mu=0.000459
Iteration 11, ||r_p||=0.000002, ||r_d||=0.000035, mu=0.000275
Iteration 12, ||r_p||=0.000001, ||r_d||=0.000011, mu=0.000112
Iteration 13, ||r_p||=0.000000, ||r_d||=0.000001, mu=0.000050
Iteration 14, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000022
Iteration 15, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000010
Iteration 16, ||r_p||=0.000000, ||r_d||=0.000000, mu=0.000004
Iteration

In [49]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = cholLap);

Iteration 1, ||r_p||=45.808803, ||r_d||=826.954312, mu=0.804767
Iteration 2, ||r_p||=24.810426, ||r_d||=447.885281, mu=0.664281


LoadError: InterruptException:

I stopped that code, because it was taking forever.

## Matlab connection

Finally, I note that if you are have Matlab, and can install Koutis's KMP solvers so that they run from Matlab, then you can also call them from Julia.  The only annoying thing about them is that they spit out a lot of diagonstics. To use them, you need to get the MATLAB.jl package working.  Then, you can use the matlab solvers like this:

In [51]:
include("/Users/spielman/Laplacians/src/matlabSolvers.jl")

>> >> >> Connected to Matlab!


matlabCmgLap

In [52]:
@time (x_ntipm_flow,s,y) = min_cost_flow(Bt',c,b,u, lapSolver = (h->matlabCmgLap(h,tol=1e-8)));

Iteration 1, ||r_p||=45.808803, ||r_d||=826.954312, mu=0.804767
>> >> >> 

Level 2. Graph vertices: 14200 
Density : 10.037
Reduction factor in vertices : 3.52106
Reduction factor in edges    : 1.75383
*****************
Level 3. Graph vertices: 5489 
Density : 21.153
Reduction factor in vertices : 2.58699
Reduction factor in edges    : 1.22752
*****************
Level 4. Graph vertices: 4736 
Density : 24.0025
Reduction factor in vertices : 1.15899
Reduction factor in edges    : 1.0214
*****************
Level 5. Graph vertices: 4638 
Density : 24.4299
Reduction factor in vertices : 1.02113
Reduction factor in edges    : 1.00327
*****************
Level 6. Graph vertices: 4609 
Density : 24.557
Reduction factor in vertices : 1.00629
Reduction factor in edges    : 1.00109
*****************
[Warning: CMG convergence may be slow due to matrix density. Future versions of
CMG will eliminate this problem.] 
[> In hierarchy (line 195)
  In cmg_dd (line 92)
  In cmg_sdd (line 41)] 
Hierarchy 

That was the fastest.  But, there wasn't a huge amount of difference between the solvers, other than the cholesky one.  We should check if there is another part of the code that is taking a lot of time.
